# Model Adversarial Training

## Import Libraries

In [1]:
import numpy as np
import os, torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import transforms
from models.project_models import FC_500_150, LeNet_CIFAR, LeNet_MNIST, Net
from utils.project_utils import get_clip_bounds, model_train, model_eval

## CIFAR-10 SSL Fix

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## Set Random Seed & Device

In [3]:
rand_seed = 100
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)

use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

## Download & Pre-Process Datasets

### MNIST Dataset

In [4]:
mnist_mean = 0.5
mnist_std = 0.5
mnist_dim = 28

mnist_min, mnist_max = get_clip_bounds(mnist_mean,
                                       mnist_std,
                                       mnist_dim)
mnist_min = mnist_min.to(device)
mnist_max = mnist_max.to(device)

mnist_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=mnist_mean,
        std=mnist_std)])

mnist_tf_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=mnist_mean,
        std=mnist_std)])

mnist_tf_inv = transforms.Compose([
    transforms.Normalize(
        mean=0.0,
        std=np.divide(1.0, mnist_std)),
    transforms.Normalize(
        mean=np.multiply(-1.0, mnist_std),
        std=1.0)])

mnist_temp = datasets.MNIST(root='datasets/mnist', train=True,
                            download=True, transform=mnist_tf_train)
mnist_train, mnist_val = random_split(mnist_temp, [50000, 10000])

mnist_test = datasets.MNIST(root='datasets/mnist', train=False,
                            download=True, transform=mnist_tf)

cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                 'dog', 'frog', 'horse', 'ship', 'truck']

### CIFAR-10 Dataset

In [5]:
cifar_mean = [0.491, 0.482, 0.447]
cifar_std = [0.202, 0.199, 0.201]
cifar_dim = 32

cifar_min, cifar_max = get_clip_bounds(cifar_mean,
                                       cifar_std,
                                       cifar_dim)
cifar_min = cifar_min.to(device)
cifar_max = cifar_max.to(device)

cifar_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=cifar_mean,
        std=cifar_std)])

cifar_tf_train = transforms.Compose([
    transforms.RandomCrop(
        size=cifar_dim,
        padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=cifar_mean,
        std=cifar_std)])

cifar_tf_inv = transforms.Compose([
    transforms.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=np.divide(1.0, cifar_std)),
    transforms.Normalize(
        mean=np.multiply(-1.0, cifar_mean),
        std=[1.0, 1.0, 1.0])])

cifar_temp = datasets.CIFAR10(root='datasets/cifar-10', train=True,
                              download=True, transform=cifar_tf_train)
cifar_train, cifar_val = random_split(cifar_temp, [40000, 10000])

cifar_test = datasets.CIFAR10(root='datasets/cifar-10', train=False,
                              download=True, transform=cifar_tf)

Files already downloaded and verified
Files already downloaded and verified


## Dataset Loaders

In [6]:
batch_size = 128
workers = 4

mnist_loader_train = DataLoader(mnist_train, batch_size=batch_size,
                                shuffle=True, num_workers=workers)
mnist_loader_val = DataLoader(mnist_val, batch_size=batch_size,
                              shuffle=False, num_workers=workers)
mnist_loader_test = DataLoader(mnist_test, batch_size=batch_size,
                               shuffle=False, num_workers=workers)

cifar_loader_train = DataLoader(cifar_train, batch_size=batch_size,
                                shuffle=True, num_workers=workers)
cifar_loader_val = DataLoader(cifar_val, batch_size=batch_size,
                              shuffle=False, num_workers=workers)
cifar_loader_test = DataLoader(cifar_test, batch_size=batch_size,
                               shuffle=False, num_workers=workers)

## Training Parameters

In [7]:
train_model = True

epochs = 50
epochs_nin = 100

lr = 0.004
lr_nin = 0.01
lr_scale = 0.5

momentum = 0.9

print_step = 5

deep_batch_size = 10
deep_num_classes = 10
deep_overshoot = 0.02
deep_max_iters = 50

deep_args = [deep_batch_size, deep_num_classes,
             deep_overshoot, deep_max_iters]

if not os.path.isdir('weights/deepfool'):
    os.makedirs('weights/deepfool', exist_ok=True)

if not os.path.isdir('weights/fgsm'):
    os.makedirs('weights/fgsm', exist_ok=True)

## FGSM Adversarial Training

### MNIST Dataset

#### LeNet Model

In [8]:
fgsm_eps = 0.5
model = LeNet_MNIST().to(device)
model.load_state_dict(torch.load('weights/clean/mnist_lenet.pth'))

if train_model:
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr * lr_scale,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, epochs,
                              mnist_loader_train, mnist_loader_val,
                              print_step, mnist_min, mnist_max,
                              fgsm_eps, is_fgsm=True)
    torch.save(model.state_dict(), 'weights/fgsm/mnist_lenet.pth')

model.load_state_dict(torch.load('weights/fgsm/mnist_lenet.pth'))
_, _ = model_eval(device, model, mnist_loader_test,
    mnist_min, mnist_max, fgsm_eps, is_fgsm=True)
_, _ = model_eval(device, model, mnist_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.4207,  Train Loss : 1.7140
      Val Acc : 0.4958,    Val Loss : 1.3642
Epoch [5]
    Train Acc : 0.6270,  Train Loss : 1.0105
      Val Acc : 0.6418,    Val Loss : 0.9728
Epoch [10]
    Train Acc : 0.7459,  Train Loss : 0.7206
      Val Acc : 0.7520,    Val Loss : 0.7109
Epoch [15]
    Train Acc : 0.8093,  Train Loss : 0.5454
      Val Acc : 0.8125,    Val Loss : 0.5351
Epoch [20]
    Train Acc : 0.8563,  Train Loss : 0.4213
      Val Acc : 0.8605,    Val Loss : 0.4018
Epoch [25]
    Train Acc : 0.8774,  Train Loss : 0.3580
      Val Acc : 0.8784,    Val Loss : 0.3546
Epoch [30]
    Train Acc : 0.8978,  Train Loss : 0.3037
      Val Acc : 0.8921,    Val Loss : 0.3112
Epoch [35]
    Train Acc : 0.9063,  Train Loss : 0.2758
      Val Acc : 0.8971,    Val Loss : 0.2954
Epoch [40]
    Train Acc : 0.9132,  Train Loss : 0.2546
      Val Acc : 0.9139,    Val Loss : 0.2554
Epoch [45]
    Train Acc : 0.9193,  Train Loss : 0.2379
      Val Acc : 0.9144,    Val Loss :

#### FC Model

In [9]:
fgsm_eps = 0.2
model = FC_500_150().to(device)
model.load_state_dict(torch.load('weights/clean/mnist_fc.pth'))

if train_model:
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr * lr_scale,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, epochs,
                              mnist_loader_train, mnist_loader_val,
                              print_step, mnist_min, mnist_max,
                              fgsm_eps, is_fgsm=True)
    torch.save(model.state_dict(), 'weights/fgsm/mnist_fc.pth')

model.load_state_dict(torch.load('weights/fgsm/mnist_fc.pth'))
_, _ = model_eval(device, model, mnist_loader_test,
    mnist_min, mnist_max, fgsm_eps, is_fgsm=True)
_, _ = model_eval(device, model, mnist_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.5398,  Train Loss : 1.3971
      Val Acc : 0.6148,    Val Loss : 1.0703
Epoch [5]
    Train Acc : 0.7183,  Train Loss : 0.7871
      Val Acc : 0.7193,    Val Loss : 0.7890
Epoch [10]
    Train Acc : 0.7648,  Train Loss : 0.6703
      Val Acc : 0.7626,    Val Loss : 0.6810
Epoch [15]
    Train Acc : 0.7891,  Train Loss : 0.6054
      Val Acc : 0.7825,    Val Loss : 0.6192
Epoch [20]
    Train Acc : 0.8074,  Train Loss : 0.5587
      Val Acc : 0.7988,    Val Loss : 0.5771
Epoch [25]
    Train Acc : 0.8201,  Train Loss : 0.5248
      Val Acc : 0.8107,    Val Loss : 0.5472
Epoch [30]
    Train Acc : 0.8305,  Train Loss : 0.4962
      Val Acc : 0.8219,    Val Loss : 0.5218
Epoch [35]
    Train Acc : 0.8382,  Train Loss : 0.4726
      Val Acc : 0.8287,    Val Loss : 0.5055
Epoch [40]
    Train Acc : 0.8458,  Train Loss : 0.4533
      Val Acc : 0.8335,    Val Loss : 0.4877
Epoch [45]
    Train Acc : 0.8536,  Train Loss : 0.4341
      Val Acc : 0.8436,    Val Loss :

### CIFAR-10 Dataset

#### Network-In-Network Model

In [10]:
fgsm_eps = 0.2
model = Net().to(device)
model.load_state_dict(torch.load('weights/clean/cifar_nin.pth'))

if train_model:
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr_nin * lr_scale,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, epochs_nin,
                              cifar_loader_train, cifar_loader_val,
                              print_step, cifar_min, cifar_max,
                              fgsm_eps, is_fgsm=True)
    torch.save(model.state_dict(), 'weights/fgsm/cifar_nin.pth')

model.load_state_dict(torch.load('weights/fgsm/cifar_nin.pth'))
_, _ = model_eval(device, model, cifar_loader_test,
    cifar_min, cifar_max, fgsm_eps, is_fgsm=True)
_, _ = model_eval(device, model, cifar_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.1074,  Train Loss : 2.4102
      Val Acc : 0.1390,    Val Loss : 2.3601
Epoch [5]
    Train Acc : 0.2046,  Train Loss : 2.0998
      Val Acc : 0.1783,    Val Loss : 2.1647
Epoch [10]
    Train Acc : 0.2423,  Train Loss : 2.0235
      Val Acc : 0.2092,    Val Loss : 2.1111
Epoch [15]
    Train Acc : 0.2617,  Train Loss : 1.9804
      Val Acc : 0.2170,    Val Loss : 2.0712
Epoch [20]
    Train Acc : 0.2756,  Train Loss : 1.9500
      Val Acc : 0.2306,    Val Loss : 2.0355
Epoch [25]
    Train Acc : 0.2884,  Train Loss : 1.9188
      Val Acc : 0.2408,    Val Loss : 2.0174
Epoch [30]
    Train Acc : 0.3004,  Train Loss : 1.8866
      Val Acc : 0.2646,    Val Loss : 1.9665
Epoch [35]
    Train Acc : 0.3081,  Train Loss : 1.8554
      Val Acc : 0.2885,    Val Loss : 1.9188
Epoch [40]
    Train Acc : 0.3250,  Train Loss : 1.8166
      Val Acc : 0.2951,    Val Loss : 1.8921
Epoch [45]
    Train Acc : 0.3386,  Train Loss : 1.7760
      Val Acc : 0.2955,    Val Loss :

#### LeNet Model

In [11]:
fgsm_eps = 0.1
model = LeNet_CIFAR().to(device)
model.load_state_dict(torch.load('weights/clean/cifar_lenet.pth'))

if train_model:
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr * lr_scale,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, epochs,
                              cifar_loader_train, cifar_loader_val,
                              print_step, cifar_min, cifar_max,
                              fgsm_eps, is_fgsm=True)
    torch.save(model.state_dict(), 'weights/fgsm/cifar_lenet.pth')

model.load_state_dict(torch.load('weights/fgsm/cifar_lenet.pth'))
_, _ = model_eval(device, model, cifar_loader_test,
    cifar_min, cifar_max, fgsm_eps, is_fgsm=True)
_, _ = model_eval(device, model, cifar_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.1752,  Train Loss : 2.2145
      Val Acc : 0.2261,    Val Loss : 2.0391
Epoch [5]
    Train Acc : 0.3121,  Train Loss : 1.8308
      Val Acc : 0.3194,    Val Loss : 1.8296
Epoch [10]
    Train Acc : 0.3517,  Train Loss : 1.7190
      Val Acc : 0.3513,    Val Loss : 1.7234
Epoch [15]
    Train Acc : 0.3759,  Train Loss : 1.6519
      Val Acc : 0.3696,    Val Loss : 1.6818
Epoch [20]
    Train Acc : 0.3870,  Train Loss : 1.6105
      Val Acc : 0.3777,    Val Loss : 1.6411
Epoch [25]
    Train Acc : 0.4004,  Train Loss : 1.5792
      Val Acc : 0.3863,    Val Loss : 1.6273
Epoch [30]
    Train Acc : 0.4063,  Train Loss : 1.5544
      Val Acc : 0.4022,    Val Loss : 1.5902
Epoch [35]
    Train Acc : 0.4172,  Train Loss : 1.5316
      Val Acc : 0.4074,    Val Loss : 1.5854
Epoch [40]
    Train Acc : 0.4227,  Train Loss : 1.5122
      Val Acc : 0.4135,    Val Loss : 1.5699
Epoch [45]
    Train Acc : 0.4294,  Train Loss : 1.4932
      Val Acc : 0.4113,    Val Loss :

## DeepFool Adversarial Training

### MNIST Dataset

#### LeNet Model

In [12]:
model = LeNet_MNIST().to(device)
model.load_state_dict(torch.load('weights/clean/mnist_lenet.pth'))

if train_model:
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr * lr_scale,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, 5,
                              mnist_loader_train, mnist_loader_val,
                              1, mnist_min, mnist_max,
                              deep_args, is_fgsm=False)
    torch.save(model.state_dict(), 'weights/deepfool/mnist_lenet.pth')

model.load_state_dict(torch.load('weights/deepfool/mnist_lenet.pth'))
_, _ = model_eval(device, model, mnist_loader_test,
    mnist_min, mnist_max, deep_args, is_fgsm=False)
_, _ = model_eval(device, model, mnist_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.0339,  Train Loss : 0.8502
      Val Acc : 0.0292,    Val Loss : 0.8463
Epoch [2]
    Train Acc : 0.0327,  Train Loss : 0.8121
      Val Acc : 0.0347,    Val Loss : 0.8027
Epoch [3]
    Train Acc : 0.0326,  Train Loss : 0.7890
      Val Acc : 0.0296,    Val Loss : 0.7875
Epoch [4]
    Train Acc : 0.0322,  Train Loss : 0.7802
      Val Acc : 0.0283,    Val Loss : 0.7802
Epoch [5]
    Train Acc : 0.0310,  Train Loss : 0.7751
      Val Acc : 0.0316,    Val Loss : 0.7775
Evaluation (DeepFool Images)
     Test Acc : 0.0308,   Test Loss : 0.7807
Evaluation (Clean Images)
     Test Acc : 0.9585,   Test Loss : 0.1559


#### FC Model

In [13]:
model = FC_500_150().to(device)
model.load_state_dict(torch.load('weights/clean/mnist_fc.pth'))

if train_model:
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr * lr_scale,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, 5,
                              mnist_loader_train, mnist_loader_val,
                              1, mnist_min, mnist_max,
                              deep_args, is_fgsm=False)
    torch.save(model.state_dict(), 'weights/deepfool/mnist_fc.pth')

model.load_state_dict(torch.load('weights/deepfool/mnist_fc.pth'))
_, _ = model_eval(device, model, mnist_loader_test,
    mnist_min, mnist_max, deep_args, is_fgsm=False)
_, _ = model_eval(device, model, mnist_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.0567,  Train Loss : 0.7843
      Val Acc : 0.0545,    Val Loss : 0.8015
Epoch [2]
    Train Acc : 0.0527,  Train Loss : 0.7819
      Val Acc : 0.0526,    Val Loss : 0.7956
Epoch [3]
    Train Acc : 0.0494,  Train Loss : 0.7759
      Val Acc : 0.0480,    Val Loss : 0.7823
Epoch [4]
    Train Acc : 0.0477,  Train Loss : 0.7708
      Val Acc : 0.0444,    Val Loss : 0.7766
Epoch [5]
    Train Acc : 0.0457,  Train Loss : 0.7684
      Val Acc : 0.0422,    Val Loss : 0.7769
Evaluation (DeepFool Images)
     Test Acc : 0.0415,   Test Loss : 0.7805
Evaluation (Clean Images)
     Test Acc : 0.9396,   Test Loss : 0.2478


### CIFAR-10 Dataset

#### Network-In-Network Model

In [14]:
model = Net().to(device)
model.load_state_dict(torch.load('weights/clean/cifar_nin.pth'))

if train_model:
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr_nin * lr_scale,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, 10,
                              cifar_loader_train, cifar_loader_val,
                              1, cifar_min, cifar_max,
                              deep_args, is_fgsm=False)
    torch.save(model.state_dict(), 'weights/deepfool/cifar_nin.pth')

model.load_state_dict(torch.load('weights/deepfool/cifar_nin.pth'))
_, _ = model_eval(device, model, cifar_loader_test,
    cifar_min, cifar_max, deep_args, is_fgsm=False)
_, _ = model_eval(device, model, cifar_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.5524,  Train Loss : 0.9408
      Val Acc : 0.0798,    Val Loss : 0.8160
Epoch [2]
    Train Acc : 0.5423,  Train Loss : 0.9364
      Val Acc : 0.0897,    Val Loss : 0.8516
Epoch [3]
    Train Acc : 0.5404,  Train Loss : 0.9389
      Val Acc : 0.0957,    Val Loss : 0.8616
Epoch [4]
    Train Acc : 0.5379,  Train Loss : 0.9459
      Val Acc : 0.1032,    Val Loss : 0.8693
Epoch [5]
    Train Acc : 0.5323,  Train Loss : 0.9540
      Val Acc : 0.1066,    Val Loss : 0.8885
Epoch [6]
    Train Acc : 0.5353,  Train Loss : 0.9525
      Val Acc : 0.1082,    Val Loss : 0.8878
Epoch [7]
    Train Acc : 0.5277,  Train Loss : 0.9617
      Val Acc : 0.1015,    Val Loss : 0.8937
Epoch [8]
    Train Acc : 0.5259,  Train Loss : 0.9628
      Val Acc : 0.1043,    Val Loss : 0.9029
Epoch [9]
    Train Acc : 0.5236,  Train Loss : 0.9682
      Val Acc : 0.1176,    Val Loss : 0.9017
Epoch [10]
    Train Acc : 0.5231,  Train Loss : 0.9674
      Val Acc : 0.1180,    Val Loss : 0.9025

#### LeNet Model

In [15]:
model = LeNet_CIFAR().to(device)
model.load_state_dict(torch.load('weights/clean/cifar_lenet.pth'))

if train_model:
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr * lr_scale,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, 5,
                              cifar_loader_train, cifar_loader_val,
                              1, cifar_min, cifar_max,
                              deep_args, is_fgsm=False)
    torch.save(model.state_dict(), 'weights/deepfool/cifar_lenet.pth')

model.load_state_dict(torch.load('weights/deepfool/cifar_lenet.pth'))
_, _ = model_eval(device, model, cifar_loader_test,
    cifar_min, cifar_max, deep_args, is_fgsm=False)
_, _ = model_eval(device, model, cifar_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.1477,  Train Loss : 1.0705
      Val Acc : 0.1622,    Val Loss : 1.2455
Epoch [2]
    Train Acc : 0.1647,  Train Loss : 1.1456
      Val Acc : 0.1624,    Val Loss : 1.2241
Epoch [3]
    Train Acc : 0.1659,  Train Loss : 1.1551
      Val Acc : 0.1637,    Val Loss : 1.2288
Epoch [4]
    Train Acc : 0.1650,  Train Loss : 1.1672
      Val Acc : 0.1640,    Val Loss : 1.2440
Epoch [5]
    Train Acc : 0.1663,  Train Loss : 1.1708
      Val Acc : 0.1682,    Val Loss : 1.2484
Evaluation (DeepFool Images)
     Test Acc : 0.1636,   Test Loss : 1.2251
Evaluation (Clean Images)
     Test Acc : 0.6780,   Test Loss : 1.2484
